In [4]:
# MHAR walk

In [5]:
#| default_exp walk

In [6]:
#| export
import torch

from typing import  Union


from mhar.polytope import Polytope, NFDPolytope

In [7]:
#| export

def sample_inner_points(X0, z):
    n, k = X0.shape
    if z > k:
        # Pad by repeating vectors from X to fill z columns
        num_repeats = z // k
        remainder = z % k
        I = torch.cat([X0] * num_repeats + [X0[:, :remainder]], dim=1)
    elif z < k:
        # Take the first z columns from X
        I = X0[:, :z]
    else:
        # z == k, I equals X
        I = X0.clone()
    return I

In [ ]:
def create_h(n,z,generator,dtype,device):
    """
    Creates a Tensor (z x n x 1) where each entry ~ N(0,1). Automatically detects the
    precision 64 bits or 32 bits.
    -------------
    :param n:   int
                Dimension of the Space where the Polytope Lives
    :param z:   int
                Padding Parameter
    :param generator:
    :param device:  String, default = cpu
                    Hardware used to make the computations and allocate the result.
                    If equal to cpu then the CPUs are used for computing the inverse.
                    If equal to cuda then the a GPU is used for computing the inverse.
    -------------
    :return:    Torch Tensor
                Tensor (z x n x 1) where each entry ~ N(0,1)Contains a tensor

    """
    if '64' in str(dtype):
        if 'cuda' == device:
            h = torch.cuda.DoubleTensor(z, n, 1).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.DoubleTensor(z, n, 1).normal_(generator=generator)
    elif '32' in str(dtype):
        if 'cuda' == device:
            h = torch.cuda.FloatTensor(z, n, 1).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.FloatTensor(z, n, 1).normal_(generator=generator)
    elif '16' in str(dtype):
        if 'cuda' == device:
            h = torch.cuda.HalfTensor(z, n, 1).normal_(generator=generator)
        elif 'cpu' == device:
            h = torch.HalfTensor(z, n, 1).normal_(generator=generator)

    return h

In [ ]:
def draw_uniform(z, generator,dtype,device='cpu'):
    """
    Creates a tensor (z x 1) where each entry ~ U(0,1). Automatically detects the
    precision 64 bits or 32 bits.
    -------------
    :param n:   int
                Dimension of the Space where the Polytope Lives
    :param z:   int
                Padding Parameter
    :param generator:
    :param device:  String, default = cpu
                    Hardware used to make the computations and allocate the result.
                    If equal to cpu then the CPUs are used for computing the inverse.
                    If equal to cuda then the a GPU is used for computing the inverse.
    -------------
    :return:    Torch Tensor
                Tensor (z x 1) where each entry ~ U(0,1)

    """

    if '64' in str(dtype):
        if 'cuda' == device:
            u = torch.cuda.DoubleTensor(z, 1).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.DoubleTensor(z, 1).uniform_(generator=generator)
    elif '32' in str(dtype):
        if 'cuda' == device:
            u = torch.cuda.FloatTensor(z, 1).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.FloatTensor(z, 1).uniform_(generator=generator)
    elif '16' in str(dtype):
        if 'cuda' == device:
            u = torch.cuda.HalfTensor(z, 1).uniform_(generator=generator)
        elif 'cpu' == device:
            u = torch.HalfTensor(z, 1).uniform_(generator=generator)

    return u
    

In [9]:
#| export

def walk(
        polytope:Union[Polytope, NFDPolytope], # Polytope Object
        X0:torch.Tensor,    # Initial Interior point(s) of dim=(n,k).     
                            #- If z > k (number of columns in X0), pad  by repeating vectors from X to fill z columns.
                            #- If z < k, take the first z columns from X0.
                            #- If z == k, equal to X0.
        z:int=1, # The number of simultaneous to be executed (padding parameter).
        T:int=1, # id-iterations, total_iid_points = T*z. Each iid iteration will burn the samples established by the thinning factor.
        warm:int=0, # Number of iid-iterations needed to warm. The walk will execute warm steps before saving the points.
        thinning:int=None, # Thinning Factor. Default O(n^3)
        device:str = 'cpu', # Deveice to use, cpu or cuda
        seed:int=None, # Seed for Pseudo-Random Number Generation
        verbosity:int=1, # Verbosity of the execution
        ) -> None:
        
    ## Check validity 
    # Device
    assert(device in ['cpu', 'cuda']), print('The device is not correctly specified: ', device,
                                        '\n Please choose cpu or cuda')
    # X0 dimension
    assert(X0.shape[0] == polytope.n)
    
    ## Set min and max values
    min_ = torch.finfo(polytope.dtype).min + 2.0
    max_ = torch.finfo(polytope.dtype).max - 2.0
    if verbosity > 1:
        print(f'Minimum number allowed {min_}')
        print(f'Maximum number allowed {max_}')
    
    
    ## Set seed
    random_gen = torch.Generator(device=device)
    if seed:
        random_gen.manual_seed(seed)
    else:
        random_gen.seed()
    
    
    ## Check Dimensions
    n = polytope.n
    mI = polytope.mI
    if isinstance(polytope, NFDPolytope) :
        mE = polytope.mE
    else:
        mE=None            
    if verbosity >=1:
        print('n: ', n, '  mI:', mI, '  mE:', mE, '  z:', z)
        
    
    ## Compute/set thinning factor
    if thinning:
        pass
    else:
        thinning = int(n * n * n)
        if verbosity >= 1:
            print('Automatic Thinning factor: ', thinning)
     
            
    ## Prepare and send Matrices
    init_x0 = sample_inner_points(X0,z).to(device)  
    polytope.send_to_device(device)
    
    
    ## Iteration Loop
    t = 1
    burned = 0
    dtype = polytope.dtype
    while t <= T:
        h = create_h(n, z, generator=random_gen, dtype=dtype,device=device)
    